In [ ]:
import gurobipy as gp

# Q1

In [ ]:
types=["longrange","mediumrange","shortrange"]

price={"longrange":67,"mediumrange":50,"shortrange":35}  #in millions
profit={"longrange":4.2,"mediumrange":3,"shortrange":2.3}
maintananceNeeded={"longrange":5/3,"mediumrange":5/3,"shortrange":1} 
maintananceAvail=40
pilots=30
budget=1500 #in millions


model=gp.Model()
x=model.addVars(types,vtype=gp.GRB.INTEGER,obj=profit,name=" ")
model.ModelSense=gp.GRB.MAXIMIZE
model.addConstr(gp.quicksum(x[t] for t in types)<=pilots)
model.addConstr(gp.quicksum(price[t]*x[t] for t in types)<=budget)
model.addConstr(gp.quicksum(maintananceNeeded[t]*x[t] for t in types)<=maintananceAvail)

model.optimize()

model.printAttr("X")
print ("total profit", model.ObjVal)

# Q2

In [ ]:
fixedCosts=[1000,950,875,850,800,700]
variableCosts=[21,23,25,24,20,26]
capacity=[500,600,750,400,600,800]
totalOrders=1800
n=len(fixedCosts)

model=gp.Model()
x=model.addVars(n,vtype=gp.GRB.INTEGER,obj=variableCosts,name="x")
y=model.addVars(n,vtype=gp.GRB.BINARY,obj=fixedCosts,name="y")
model.addConstr(gp.quicksum(x[i] for i in range(n))==totalOrders)
model.addConstrs(x[i]<=capacity[i]*y[i] for i in range(n))
model.optimize()
model.printAttr("X")
print ("total profit", model.ObjVal)

# Q3

In [ ]:
shows=["C","D","L","J","B","N","F","M","U"]
revenues={"C":6,"D":10,"L":9,"J":4,"B":5,"N":2,"F":6,"M":7,"U":8}
publicInterest=["L","N","F","U"]
violence=["C","D","L","J","N"]
comedy=["C","B","M"]
drama=["D","L","J","F"]
bigM=len(shows) #using bigM to deactivate constraints. 
                #Being lazy always use the same bigM
model=gp.Model()
model.ModelSense=gp.GRB.MAXIMIZE
x=model.addVars(shows,obj=revenues,vtype=gp.GRB.BINARY,name="Show")
#exactly 5 shows
model.addConstr(gp.quicksum(x[s] for s in shows)==5) 
#constraint 1
model.addConstr(gp.quicksum(x[p] for p in publicInterest)>=
                gp.quicksum(x[v] for v in violence))
#constraint 2
model.addConstr(x["J"]+x["L"]>= x["F"])
#constraint 3
model.addConstr(x["F"]+x["U"]<= 1)
#contraint 4
y=model.addVar(vtype=gp.GRB.BINARY,name="at_least_2_commedy")
model.addConstr(gp.quicksum(x[c] for c in comedy)<=1+bigM*y)
model.addConstr(gp.quicksum(x[d] for d in drama)>=y)
#constraint 5, notice the objective coefficient in the z indicator 
z=model.addVar(vtype=gp.GRB.BINARY,obj=-4,name="at_least_3_violence")
model.addConstr(gp.quicksum(x[v] for v in violence)<=2+bigM*z)

model.optimize()
print("optimal value",model.objval)
model.printAttr("X")



# Q4

In [ ]:
import pandas as pd
df=pd.read_csv("Q4_data.csv")
towns=list(df.Town)
distances={}
for t1 in towns:
    for t2 in towns:
        distances[(t1,t2)]=float(df[df.Town==t1][t2])

model=gp.Model()
x=model.addVars(towns,vtype=gp.GRB.BINARY,obj=1,name=" ")
model.addConstrs(
    gp.quicksum(x[t1] for t1 in towns if (distances[(t1,t2)]<=10)) >=1 
        for t2 in towns)
model.optimize()
print("Total Machines required",model.objval)
model.printAttr("X")

# Q5a

In [ ]:
totalRows=40
ticketTypes=["First","Business","Economy"]
profitPerTicket={"First":4,"Business":3,"Economy":1}
seatsPerRow={"First":2,"Business":4,"Economy":6}

model=gp.Model()
x=model.addVars(ticketTypes,vtype=gp.GRB.INTEGER,lb=3,name="Rows")
model.setObjective(gp.quicksum(profitPerTicket[t]*seatsPerRow[t]*x[t]
                              for t in ticketTypes),gp.GRB.MAXIMIZE)
model.addConstr(gp.quicksum(x[t]for t in ticketTypes)==totalRows)
economySeats=seatsPerRow["Economy"]*x["Economy"]
otherSeats=gp.quicksum(seatsPerRow[t]*x[t] 
                       for t in ticketTypes if t!="Economy")
model.addConstr(economySeats>=2*otherSeats)

model.optimize()
print("optimal value",model.objval)
model.printAttr("X")

# Q5b

In [ ]:
totalRows=40
ticketTypes=["First","Business","Economy"]
profitPerTicket={"First":4,"Business":3,"Economy":1}
seatsPerRow={"First":2,"Business":4,"Economy":6}
weekendDemands={"First":7,"Business":18,"Economy":120}
workingDayDemands={"First":4,"Business":35,"Economy":80}
workingDayFlights=10
weekendFlights=4

model=gp.Model()
x=model.addVars(ticketTypes,vtype=gp.GRB.INTEGER,lb=3,name="Rows")
wdt=model.addVars(ticketTypes,vtype=gp.GRB.INTEGER,name="WD")#working day
sst=model.addVars(ticketTypes,vtype=gp.GRB.INTEGER,name="SS")#saturdaysunday day

model.setObjective(gp.quicksum(profitPerTicket[t]*(workingDayFlights*wdt[t]+weekendFlights*sst[t])   
                              for t in ticketTypes),gp.GRB.MAXIMIZE)
model.addConstr(gp.quicksum(x[t]for t in ticketTypes)==totalRows)
economySeats=seatsPerRow["Economy"]*x["Economy"]
otherSeats=gp.quicksum(seatsPerRow[t]*x[t] 
                       for t in ticketTypes if t!="Economy")
model.addConstr(economySeats>=2*otherSeats)
model.addConstrs(wdt[t]<=workingDayDemands[t] for t in ticketTypes)
model.addConstrs(wdt[t]<=seatsPerRow[t]*x[t] for t in ticketTypes)
model.addConstrs(sst[t]<=weekendDemands[t] for t in ticketTypes)
model.addConstrs(sst[t]<=seatsPerRow[t]*x[t] for t in ticketTypes)

model.optimize()
print("optimal value",model.objval)
model.printAttr("X")